In [5]:
# import sys
# !{sys.executable} -m  pip install underthesea

In [6]:
import pandas as pd
import numpy as np
import underthesea as ud
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv('final.csv', encoding = 'utf-8')

In [8]:
#stopwords = pd.read_table('vietnamese-stopwords.txt', sep = '\\t', header = None, names = ['word'])
#stopwords

In [9]:
df.head()

,list_id,ad_id,subject,body,price,mc_brand_name,brand,mc_model_name,model,aa_date_time,sold_date,duration,lead,spending,regdate,mileage,region,mc_type,mc_capacity
0,48871389,72344446,Cần bán xe HonDa Wave s 100,Cần bán xe HonDa Wave S 100. Xe do mình đứng t...,8700000,Honda,1,Wave,22,2018-08-01 07:04:05.8741,2018-08-01 18:22:20.82729,11,78,0,2008,NaN,2,2.0,2.0
1,48871466,72344805,Cần bán xe như hình còn đăng kí nhé,Có fix nhẹ cho anh em nhiệt tình muốn mua nhé,8000000,Honda,1,67,1,2018-08-01 07:05:37.217184,2018-09-22 11:32:55.767056,1252,37,0,1997,NaN,12,3.0,5.0
2,48871512,72345802,Future 125Fi 214 C.chủ Zin A.Z BSTP . Mới 95%,Lên đời Cần ra đi e Future 125Fi 2014 Xe chạy...,19800000,Honda,1,Future,13,2018-08-01 07:06:54.144895,2018-08-02 11:36:03.65795,28,21,0,2014,NaN,13,2.0,3.0
3,48872556,72346345,Cần thanh lý em sirius,"ai mua liên hệ giúp em nha ,sdt của em",8000000,Yamaha,2,Sirius,27,2018-08-01 07:34:41.045579,2018-08-01 23:19:27.53107,16,18,0,2007,NaN,13,2.0,NaN
4,48872805,72344763,Vespa Lx 125ie,Do nhu cầu đổi xe mới nên bán lại . Ai cần liê...,18500000,Piaggio,3,LX,79,2018-08-01 07:42:17.468578,2018-08-01 17:07:44.254137,10,9,0,2011,NaN,13,1.0,2.0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50724 entries, 0 to 50723
Data columns (total 19 columns):
list_id          50724 non-null int64
ad_id            50724 non-null int64
subject          50724 non-null object
body             50721 non-null object
price            50724 non-null int64
mc_brand_name    50689 non-null object
brand            50724 non-null int64
mc_model_name    50362 non-null object
model            50724 non-null int64
aa_date_time     50724 non-null object
sold_date        50724 non-null object
duration         50724 non-null int64
lead             50724 non-null int64
spending         50724 non-null int64
regdate          50724 non-null int64
mileage          42458 non-null float64
region           50724 non-null int64
mc_type          50722 non-null float64
mc_capacity      44227 non-null float64
dtypes: float64(3), int64(10), object(6)
memory usage: 7.4+ MB


In [11]:
#Chuyen gia tri null cua bien dung tich xi lanh = 6
df['mc_capacity'] = df['mc_capacity'].fillna(value = 6).reset_index(drop = True)
#Bo cac ad co lead tu 50 tro len
df = df[df.lead < 50].reset_index(drop = True)

In [12]:
#Bo cac thong tin khong can thiet
df_new = df.drop(['mc_brand_name', 'mc_model_name', 'ad_id', 'list_id', 'aa_date_time', 'sold_date', 'lead'], axis = 1)
#Bo cac ad co spending tu 40k tro len
df_new = df_new[df_new.spending < 40000].reset_index(drop = True)
#Dua bien gia ve thang gia tri nho hon
df_new['price'] = df_new['price']/1000000
#Bo cac ad co gia tu 60tr tro len
df_new = df_new[df_new.price < 60].reset_index(drop = True)
#Sua lai bien brand lay 5 dong xe pho bien nhat, cac dong xe con lai gop chung thanh 1 nhom 6
df_new['brand'] = pd.Series([x if x <= 5 else 6 for x in df_new.brand])
#Bo het cac gia tri null
df_new = df_new.dropna().reset_index(drop = True)
#Tao dic cac model ban nhieu nhat, giu lai cac model co hon 100 ad, cac model con lai cho vao nhom 0
model_dict = dict(df_new.model.value_counts())
df_new['model'] = pd.Series([x if model_dict[x] > 500 else 0 for x in df_new.model])

In [13]:
df_new.head()

,subject,body,price,brand,model,duration,spending,regdate,mileage,region,mc_type,mc_capacity
0,Honda Wave nhật bs 61 zin nguyên bản,wave nhật 2007 bs 61 máy cực chất .xe đời...,10.3,1,22,90,35000,2007,45.0,13,2.0,2.0
1,"Future GT, đồng hồ điện tư","Xe zin ngay chủ.Ai cần liên hệ,miên tiếp tin ...",22.0,2,0,1,0,2017,800000.0,2,1.0,3.0
2,Exciter Rc 150 Trắng đỏ xe đẹp chính chủ,Gia đình đổi xe nên cần bán hay giao lưu xe kh...,34.5,2,25,13,35000,2015,17000.0,13,3.0,6.0
3,"cần tiền gấp ra đi em ex 2018 chính chủ,biển s...",cần tiền nênh ra đi con xe còn mới.zin từ tron...,37.0,2,25,524,0,2018,9000.0,13,3.0,3.0
4,Cần tiền bán gấp,xe mới mua hồi qua tết 2018 càn tiền bán gấp g...,16.0,2,27,954,0,2018,30000.0,10,2.0,3.0


In [14]:
df_new.astype('object').describe()

,subject,body,price,brand,model,duration,spending,regdate,mileage,region,mc_type,mc_capacity
count,32564,32564,32564.0,32564,32564,32564,32564,32564,32564.0,32564,32564.0,32564.0
unique,28385,32149,937.0,6,16,1365,11,40,3822.0,13,3.0,6.0
top,Honda Wave,xe nhà đang đi nay muốn bán xe nhà đi cẩn thận...,10.0,1,0,1,0,2010,20000.0,13,2.0,3.0
freq,185,54,654.0,16428,8931,2001,23939,3288,1773.0,15880,16635.0,21182.0


In [15]:
#x = df_new.lead
#3IQR = x.quantile(0.75) - x.quantile(0.25)
#(x.quantile(0.25) - 1.5*IQR, 1.5*IQR + x.quantile(0.75))

In [16]:
#categorize duration
def cate_duration(x):
    if x >= 0 and x <= 12:
        value = '00_0.5_ngay'
    elif x > 12 and x <= 48:
        value = '01_02_ngay'
    elif x > 48 and x <= 168:
        value = '03_07_ngay'
    else:
        value = 'hon_1_tuan'
    return(value)
#categorize spending
def cate_spending(x):
    if x == 0:
        value = '00'
    elif x <= 20000:
        value = '20k'
    else:
        value = '40k'
    return(value)
#categorize mileage
def cate_mileage(x):
    if x >= 0 and x <= 5000 :
        value = '05k'
    elif x > 5000 and x <= 15000:
        value = '05k_15k'
    elif x > 15000 and x <= 30000:
        value = '15k_30k'
    elif x > 30000 and x <= 60000:
        value = '30k_60k'
    elif x > 60000:
        value = '60k'
    else:
        value = 'None'
    return value

In [17]:
df_new['duration'] = df_new.duration.apply(lambda x: cate_duration(x))
df_new['spending'] = df_new.spending.apply(lambda x: cate_spending(x))
df_new['mileage'] = df_new.mileage.apply(lambda x: cate_mileage(x))
df_new['regdate'] = df_new['regdate'].astype('str')

In [18]:
df_new.head()

,subject,body,price,brand,model,duration,spending,regdate,mileage,region,mc_type,mc_capacity
0,Honda Wave nhật bs 61 zin nguyên bản,wave nhật 2007 bs 61 máy cực chất .xe đời...,10.3,1,22,03_07_ngay,40k,2007,05k,13,2.0,2.0
1,"Future GT, đồng hồ điện tư","Xe zin ngay chủ.Ai cần liên hệ,miên tiếp tin ...",22.0,2,0,00_0.5_ngay,00,2017,60k,2,1.0,3.0
2,Exciter Rc 150 Trắng đỏ xe đẹp chính chủ,Gia đình đổi xe nên cần bán hay giao lưu xe kh...,34.5,2,25,01_02_ngay,40k,2015,15k_30k,13,3.0,6.0
3,"cần tiền gấp ra đi em ex 2018 chính chủ,biển s...",cần tiền nênh ra đi con xe còn mới.zin từ tron...,37.0,2,25,hon_1_tuan,00,2018,05k_15k,13,3.0,3.0
4,Cần tiền bán gấp,xe mới mua hồi qua tết 2018 càn tiền bán gấp g...,16.0,2,27,hon_1_tuan,00,2018,15k_30k,10,2.0,3.0


In [19]:
#label ouput, 1: ban trong nua ngay, 0: ban tu 1 ngay tro len
df_new['duration'] = pd.Series([1 if x == '00_0.5_ngay' else 0 for x in df_new.duration])
df_new.head()

,subject,body,price,brand,model,duration,spending,regdate,mileage,region,mc_type,mc_capacity
0,Honda Wave nhật bs 61 zin nguyên bản,wave nhật 2007 bs 61 máy cực chất .xe đời...,10.3,1,22,0,40k,2007,05k,13,2.0,2.0
1,"Future GT, đồng hồ điện tư","Xe zin ngay chủ.Ai cần liên hệ,miên tiếp tin ...",22.0,2,0,1,00,2017,60k,2,1.0,3.0
2,Exciter Rc 150 Trắng đỏ xe đẹp chính chủ,Gia đình đổi xe nên cần bán hay giao lưu xe kh...,34.5,2,25,0,40k,2015,15k_30k,13,3.0,6.0
3,"cần tiền gấp ra đi em ex 2018 chính chủ,biển s...",cần tiền nênh ra đi con xe còn mới.zin từ tron...,37.0,2,25,0,00,2018,05k_15k,13,3.0,3.0
4,Cần tiền bán gấp,xe mới mua hồi qua tết 2018 càn tiền bán gấp g...,16.0,2,27,0,00,2018,15k_30k,10,2.0,3.0


In [20]:
#df_new.to_csv('final_preprocess.csv')

In [21]:
#def remove(text):
 #   rm0 = str(text).lower()
  #  rm1 = re.sub('[0-9]', '', str(rm0))
   # rm2 = re.sub('[!@#$%\.\-\+//():]', '', str(rm1))
    #rm3 = ud.word_tokenize(str(rm2), format='text')
    #return rm3

In [22]:
#df_new['subject'] = df_new.subject.apply(remove)
#df_new['body'] = df_new.body.apply(remove)
#stopwords['word'] = stopwords.word.apply(remove)
#stopwords = stopwords.word.values.tolist()
#df_1['mc_model_name'] = df_1.mc_model_name.apply(remove_space)

In [23]:
#stopwords

In [24]:
X = df_new.drop('duration', axis = 1)
y = df_new['duration']

In [25]:
def prepare(X, y, test_size = 0.2, new_case = False, X_new = None):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    
    #tfidf_body = TfidfVectorizer(max_features=1000).fit(X_train.body)
    #tfidf_subject = TfidfVectorizer(max_features=5000).fit(X_train.subject)
    onehot_brand = OneHotEncoder().fit(X_train.brand.values.reshape(-1,1))
    onehot_model = OneHotEncoder().fit(X_train.model.values.reshape(-1,1))
    onehot_spending = OneHotEncoder().fit(X_train.spending.values.reshape(-1,1))
    onehot_regdate = OneHotEncoder().fit(X_train.regdate.values.reshape(-1,1))
    onehot_mileage = OneHotEncoder().fit(X_train.mileage.values.reshape(-1,1))
    onehot_region =  OneHotEncoder().fit(X_train.region.values.reshape(-1,1))
    onehot_mctype =  OneHotEncoder().fit(X_train.mc_type.values.reshape(-1,1))
    onehot_mccapa =  OneHotEncoder().fit(X_train.mc_capacity.values.reshape(-1,1))
    

    #Convert
    if new_case == False:
        #X_train_body = tfidf_subject.transform(X_train.body).toarray()
        #X_test_body = tfidf_subject.transform(X_test.body).toarray()
        
        #X_train_subject = tfidf_subject.transform(X_train.subject).toarray()
        #X_test_subject = tfidf_subject.transform(X_test.subject).toarray()

        X_train_brand = onehot_brand.transform(X_train.brand.values.reshape(-1,1)).toarray()
        X_test_brand = onehot_brand.transform(X_test.brand.values.reshape(-1,1)).toarray()

        X_train_model = onehot_model.transform(X_train.model.values.reshape(-1,1)).toarray()
        X_test_model = onehot_model.transform(X_test.model.values.reshape(-1,1)).toarray()
        
        X_train_spending = onehot_spending.transform(X_train.spending.values.reshape(-1,1)).toarray()
        X_test_spending = onehot_spending.transform(X_test.spending.values.reshape(-1,1)).toarray()
        
        X_train_regdate = onehot_regdate.transform(X_train.regdate.values.reshape(-1,1)).toarray()
        X_test_regdate = onehot_regdate.transform(X_test.regdate.values.reshape(-1,1)).toarray()
        
        X_train_mileage = onehot_mileage.transform(X_train.mileage.values.reshape(-1,1)).toarray()
        X_test_mileage = onehot_mileage.transform(X_test.mileage.values.reshape(-1,1)).toarray()
        
        X_train_region = onehot_region.transform(X_train.region.values.reshape(-1,1)).toarray()
        X_test_region = onehot_region.transform(X_test.region.values.reshape(-1,1)).toarray()
        
        X_train_mctype = onehot_mctype.transform(X_train.mc_type.values.reshape(-1,1)).toarray()
        X_test_mctype = onehot_mctype.transform(X_test.mc_type.values.reshape(-1,1)).toarray()
        
        X_train_mccapa = onehot_mccapa.transform(X_train.mc_capacity.values.reshape(-1,1)).toarray()
        X_test_mccapa = onehot_mccapa.transform(X_test.mc_capacity.values.reshape(-1,1)).toarray()
        

        
        X_train_full = np.concatenate([X_train_mileage,X_train_brand, X_train_model, X_train_spending, X_train_regdate, X_train_region, X_train_mctype, X_train_mccapa], axis = 1)
        X_test_full = np.concatenate([X_test_mileage,X_test_brand, X_test_model, X_test_spending, X_test_regdate, X_test_region, X_test_mctype, X_test_mccapa], axis = 1)
        
        #X_train_full = np.concatenate([X_train_body,X_train_subject,X_train_brand, X_train_model, X_train_spending, X_train_regdate, X_train_region, X_train_mctype, X_train_mccapa], axis = 1)
        #X_test_full = np.concatenate([X_test_body,X_test_subject,X_test_brand, X_test_model, X_test_spending, X_test_regdate, X_test_region, X_test_mctype, X_test_mccapa], axis = 1)
        y_train_full = y_train
        y_test_full = y_test

        return (X_train_full, X_test_full, y_train_full, y_test_full)

In [26]:
X_train, X_test, y_train, y_test = prepare(X,y)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a Lab

In [27]:
from sklearn.linear_model import LogisticRegression
lgmod = LogisticRegression()
lgmod.fit(X_train, y_train)
y_pred = lgmod.predict(X_test)
lgmod.score(X_test, y_test)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7059726700445264

In [28]:
y_pred = (1 - (lgmod.predict_proba(X_test)[:,0] > 0.63).astype('int64'))
y_pred

array([1, 0, 1, ..., 0, 0, 1])

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.76      0.76      4590
           1       0.42      0.43      0.43      1923

   micro avg       0.66      0.66      0.66      6513
   macro avg       0.59      0.59      0.59      6513
weighted avg       0.66      0.66      0.66      6513



In [30]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[3470 1120]
 [1100  823]]


In [31]:
from sklearn.model_selection import cross_val_score
cv_10 = cross_val_score(LogisticRegression(), X_train, y_train, cv = 10)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [32]:
np.mean(cv_10)

0.6955970497729292

In [33]:
from sklearn.ensemble import RandomForestClassifier
rfmod = RandomForestClassifier(n_estimators=200, max_depth=20)
rfmod.fit(X_train, y_train)
y_pred = rfmod.predict(X_test)
rfmod.score(X_test, y_test)

0.7001381851681253

In [34]:
y_pred = (1 - (rfmod.predict_proba(X_test)[:,0] > 0.61).astype('int64'))
y_pred

array([1, 0, 0, ..., 0, 1, 1])

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.80      0.77      4590
           1       0.41      0.33      0.37      1923

   micro avg       0.66      0.66      0.66      6513
   macro avg       0.57      0.57      0.57      6513
weighted avg       0.64      0.66      0.65      6513



In [36]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

[[3657  933]
 [1281  642]]


In [37]:
import pickle


In [44]:
filename = './finalized_model.sav'
s = pickle.dumps(rfmod, open(filename, 'wb'))


TypeError: an integer is required (got type _io.BufferedWriter)

In [41]:
print('hihi', s)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

